<a href="https://colab.research.google.com/github/dpatinoecheverri/ENV717_S23/blob/main/Assignments/A06/ENV717_A6_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Assignment 6
##ENV717- Spring 2023
---

Instructor: [Dalia Patino-Echeverri](mailto:dalia.patino@duke.edu)

Author(s): [Dalia Patino-Echeverri](mailto:dalia.patino@duke.edu), [Mauricio Hernandez](mailto:mmh54@duke.edu)

[Github repository](https://github.com/dpatinoecheverri/ENV717_S23)

# 3 (20%) Unit commitment problem (UCP) :
Consider a small balancing authority that needs to schedule three thermal generators for the next six hours.  Formulate and solve the Unit Commitment problem, disregarding the ramp-feasibility of reserves and min up time and min down time requirements, and assuming that:

The generators parameters are as follows:


| **Generating Unit** | **$C_g^F$** | **$C_g^{SD}$** | **$C_g^{SU}$** | **$C_g^V$** | **$P_g^{max}$** | **$P_g^{Pmin}$** | **$R_g^{D}$** | **$R_g^{SD}$** | **$R_g^{SU}$** | **$R_g^U$** | **$V_g^U$**       | **$V_g^D$**       |
|---------------------|----------|----------|------------|----------|------------|-----------|-----------|-----------|------------|-----------|----------|----------|
| 1                   |     0    |     0    |     800    |     5    |     300    |     80    |     30    |     30    |     100    |     50    |     3    |     2    |
| 2                   | 0        | 0        | 500        | 15       | 200        | 50        | 40        | 40        | 70         | 60        | 2        | 2        |
| 3                   | 0        | 0        | 250        | 30       | 100        | 30        | 50        | 50        | 40         | 70        | 1        | 2        |

<font size ="2">
Table 1: Technical characteristics of generators., where
$C_g^F$ is the Online (fixed) cost of generating unit g (\$/h), $C_g^{SD}$ is the shut-down cost of generating unit g (\$), $C_g^{SU}$ is the Start-up cost of generating unit g (\$),
$C_g^V$ is the Variable cost of generating unit g (\$/MWh),
$P_g^{max}$ is the Power generation capacity of unit g (MW),
$P_g^{Pmin}$, is the minimum power output of unit g (MW),
$R_g^{D}$ is the ramping-down limit of generating unit g (MW/h),
$R_g^{SD}$ is the shut-down ramping limit of generating unit g (MW/h),
$R_g^{SU}$ is the start-up ramping limit of generating unit g(MW/h), 
$R_g^U$, is the ramping-up limit of generating unit g (MW/h),
$V_g^U$, is the minimum number of time periods the unit must run after starting up,
$V_g^D$, is theinimum number of periods the unit must be off after shutting down.
</font>

---

The initial operating conditions are:

| Generating Unit | **$$u_{g,0}$$** | **$$p_{g,0}$$**   | **$M_{g}^U$** | **$M_{g}^U$** |
|-----------------:|-------:|-----:|---:|---:|
| 1               |   1   | 130 | 2 | 0 |
| 2               |   0   | 0   | 0 | 0 |
| 3               |   0   | 0   | 0 | 0 |

<font size ="2">
where,
*$u_{g,0}$* is the On/off status of generator g at time t=0; 
*$p_{g,0}$* is the power generation of generator g at time t=0 (MW); 
*$M_{g}^U$* is the number of time periods that generator g is required to be online at the start of the planning horizon;  *$M_{g}^U$* is the number of time periods that generator g is required to be offline at the start of the planning horizon.
</font>

---
The demand and reserve requirements are (MW):

| Time period t  | 1   | 2   | 3   | 4   | 5   | 6   |
|----------------|-----|-----|-----|-----|-----|-----|
| $p_t^D$              | 240 | 250 | 200 | 170 | 230 | 190 |
| $p_t^R$              | 10  | 10  | 10  | 10  | 10  | 10  |

In [ ]:
import shutil
import sys
import os
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import numpy as np

if not shutil.which("pyomo"):
  if "google.colab" in sys.modules:
    !pip install -q pyomo
    assert(shutil.which("pyomo"))
  else:
    try:
      !conda install -c conda-forge pyomo
    except:
      pass

if not (shutil.which("cbc") or os.path.isfile("cbc")):
  !apt-get install -y -qq coinor-cbc
  assert(shutil.which("cbc"))
  if "google.colab" in sys.modules:
    #Install the CBC Solver in Colab
    !apt-get install -y -qq coinor-cbc
  else:
    try:
      # installation if not in Colab
      !conda install -c conda-forge coincbc
    except:
      pass

#Install the CPLEX Solver
#!pip install cplex
from pyomo.environ import *

Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 128126 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.11.4+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.11.4+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.108.6+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.108.6+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.17.5+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.17.5+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.60.3+repack1-2_amd64.deb ...
Unpacking coinor-libcgl1 (0.60.3+repack1-2) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to unpack .../4-coinor-libcbc3_2.10.3+repack1-1build1_amd64.deb ...
Unpacking coinor-libcbc3 (2.10.3+repack1-1build1) ...
Sel

In [ ]:
#Connect to google drive
from google.colab import drive
drive.mount('/content/gdrive')
#save path
#access path where notebook is stored
path = "/content/gdrive/MyDrive/ENV717-S2023/A06/"
#Change the working directory to the folder
os.chdir(path)

Mounted at /content/gdrive


In [ ]:
#Read input data
#from generators tab and skip the first row
df_gens = pd.read_excel(io="A6Q2_data.xlsx", sheet_name="GeneratorsData", skiprows=1)
#from demand tab and skip the first row 
df_demand = pd.read_excel(io="A6Q2_data.xlsx", sheet_name="DemandData", skiprows=1) 
#read data from reserves tab and skip 
df_reserves = pd.read_excel(io="A6Q2_data.xlsx", sheet_name="ReservesData", skiprows=1) 

df_gens.head()

,FixedCost,SDCost,SUCost,VarCost,MaxGen,MinGen,RampDown,RampSD,RampSU,RampUp,MinUpTime,MinDownTime,ReqUp,ReqDown,InitialStatus,InitialGen
0,0,0,800,5,300,80,30,30,100,50,3,2,2,0,1,130
1,0,0,500,15,200,50,40,40,70,60,2,2,0,0,0,0
2,0,0,250,30,100,30,50,50,40,70,1,2,0,0,0,0


In [ ]:
#See size of the generators and periods sets
NumGens=len(df_gens)
NumPeriods=len(df_demand)

print('There are {} generators and {} hours.'.format(NumGens, NumPeriods))

There are 3 generators and 6 hours.


In [ ]:
#Define Indexes
#This array has 2 components, in Python the indeces start at zero
N = np.array([n for n in range(0, NumGens)]) 
T = np.array([t for t in range(0, NumPeriods)])

#Need this T2 index to write the ramp-up constraints
T2 = np.array([t2 for t2 in range(1, NumPeriods)]) 
print(T2)

[1 2 3 4 5]


In [ ]:
##Declare the generator's parameters and assign them the data we read above
FixCost = df_gens['FixedCost'].to_numpy()
SDCost = df_gens['SDCost'].to_numpy()
SUCost = df_gens['SUCost'].to_numpy()
VarCost = df_gens['VarCost'].to_numpy()
PMax = df_gens['MaxGen'].to_numpy()
PMin = df_gens['MinGen'].to_numpy()
RampUp = df_gens['RampUp'].to_numpy()
RampDown = df_gens['RampDown'].to_numpy()
SURamp = df_gens['RampSU'].to_numpy()
SDRamp = df_gens['RampSD'].to_numpy()
MinUpTime = df_gens['MinUpTime'].to_numpy()
MinDownTime = df_gens['MinDownTime'].to_numpy()
ReqOn = df_gens['ReqUp'].to_numpy()
ReqOff = df_gens['ReqDown'].to_numpy()
InitialOnOff = df_gens['InitialStatus'].to_numpy()
InitialGen = df_gens['InitialGen'].to_numpy()

##Declare the system's parameters and assign them the data we read
Demand = df_demand['Demand'].to_numpy()
ReqReserves = df_reserves['Reserves'].to_numpy()

### Defining the UCP

In [ ]:
#UC Model Definition
def Unit_Commitment():
    #Initializing the model
    m = ConcreteModel()
    
    # Index definition
    m.N = Set(initialize=N)
    m.T = Set(initialize=T)
    m.T2 = Set(initialize=T2)
    
    # Variables definition
    m.p = Var(m.N, m.T, domain=NonNegativeReals) #Power generation
    m.u = Var(m.N, m.T, domain=Binary) # On-off variable
    m.y = Var(m.N, m.T, domain=Binary) # Start-up variable
    m.z = Var(m.N, m.T, domain=Binary) # Shut-down variable
    
    # Objective Function Definition
    def SysCost(m):
        Fix_cost = sum(m.u[n,t]*FixCost[n] for n in m.N for t in m.T)
        Variable_cost = sum(m.p[n,t]*VarCost[n] for n in m.N for t in m.T)
        Start_cost = sum(m.y[n,t]*SUCost[n] for n in m.N for t in m.T)
        Shut_cost = sum(m.z[n,t]*SDCost[n] for n in m.N for t in m.T)
        return Fix_cost + Variable_cost + Start_cost + Shut_cost
    
    m.System_cost=Objective(rule=SysCost, sense=minimize)
    
    # Defining constraints
    # Demand constraint
    def Demand_const(m,t):
        return sum(m.p[n,t] for n in N) == Demand[t]
    m.DemandConstraint=Constraint(m.T, rule=Demand_const) 
    
    # Minimum generator capacity constraint
    def PMin_const(m, n, t):
        return PMin[n]*m.u[n,t] <= m.p[n,t]
    m.PMinConstraint=Constraint(m.N, m.T, rule=PMin_const) 
    
    # Maximum generator capacity constraint
    def PMax_const(m, n, t):
        return PMax[n]*m.u[n,t] >= m.p[n,t]
    m.PMaxConstraint=Constraint(m.N, m.T, rule=PMax_const) 
    
    # On-off logic for t=0
    def OnOff0_const(m, n):
        return m.y[n,0]-m.z[n,0] == m.u[n,0]-InitialOnOff[n]
    m.OnOff0Constraint=Constraint(m.N, rule=OnOff0_const) 
    
    # On-off logic for all times other than t=0
    def OnOff_const(m, n, t2):
        return m.y[n,t2]-m.z[n,t2] == m.u[n,t2]-m.u[n,t2-1]
    m.OnOffConstraint=Constraint(m.N, m.T2, rule=OnOff_const)
    
    # Ramp up constraint for t=0
    def RampUp0_const(m, n):
        return m.p[n,0]-InitialGen[n] <= InitialOnOff[n]*RampUp[n]+m.y[n,0]*SURamp[n]
    m.RampUp0Constraint=Constraint(m.N, rule=RampUp0_const)
    
    # Ramp up constraint for all times other than t=0
    def RampUp_const(m, n, t2):
        return m.p[n,t2]-m.p[n,t2-1] <= m.u[n,t2-1]*RampUp[n]+m.y[n,t2]*SURamp[n]
    m.RampUpConstraint=Constraint(m.N, m.T2, rule=RampUp_const)
    
    # Ramp down constraint for t=0
    def RampDown0_const(m, n):
        return InitialGen[n]-m.p[n,0] <= m.u[n,0]*RampDown[n]+m.z[n,0]*SDRamp[n]
    m.RampDown0Constraint=Constraint(m.N, rule=RampDown0_const)
    
    # Ramp down constraint for all times other than t=0
    def RampDown_const(m, n, t2):
        return m.p[n,t2-1]-m.p[n,t2] <= m.u[n,t2]*RampDown[n]+m.z[n,t2]*SDRamp[n]
    m.RampDownConstraint=Constraint(m.N, m.T2, rule=RampDown_const)
    
    #Start-up and shut-down logic constraint
    def StartShut_const(m, n, t):
        return m.y[n,t] + m.z[n,t] <= 1
    m.StartShutConstraint=Constraint(m.N, m.T, rule=StartShut_const)
    
    # Reliability constraint (Max capacity should be higher or equal than the sum 
    # of demand and reserves)
    def Reliability_const(m, t):
        return sum(m.u[n,t]*PMax[n] for n in N) >= Demand[t] + ReqReserves[t]
    m.ReliabilityConstraint=Constraint(m.T, rule=Reliability_const)
    
    return m

### Solving the UCP

In [ ]:
#Call Unit Commitment function and pass model to variable m
m = Unit_Commitment()

#Create concrete model and solve Unit Commitment problem
opt = SolverFactory('cbc', executable='/usr/bin/cbc')
opt.solve(m).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 10300.0
  Upper bound: 10300.0
  Number of objectives: 1
  Number of constraints: 112
  Number of variables: 66
  Number of binary variables: 54
  Number of integer variables: 54
  Number of nonzeros: 30
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.02
  Wallclock time: 0.03
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and 

## a.	
Present a table with your solution specifying for all generators and time periods the following: 
1. on-off status, 
2. start-up status, 
3. Shutdown status, 
4. Power generation,



In [ ]:
#Printing all solutions 

print('System cost = ${:}'.format(m.System_cost()))
print("--------------------------------------------")
print('Commitment and Generation of each Unit n at Period t')
for n in N:
    for t in T:
        print('''n ={0:2d} \t t ={1:2d} \t {2}={3:.0f} \t {4}={5:.0f} \t {6}={7:.0f} \t {8}={9:.2f}'''.format(n+1, t+1, 
                                                m.u[n,t], 
                                                abs(m.u[n,t]()), 
                                                m.y[n,t], 
                                                abs(m.y[n,t]()), 
                                                m.z[n,t], 
                                                abs(m.z[n,t]()), 
                                                m.p[n,t], 
                                                m.p[n,t]()))

System cost = $10300.0
--------------------------------------------
Commitment and Generation of each Unit n at Period t
n = 1 	 t = 1 	 u[0,0]=1 	 y[0,0]=0 	 z[0,0]=0 	 p[0,0]=180.00
n = 1 	 t = 2 	 u[0,1]=1 	 y[0,1]=0 	 z[0,1]=0 	 p[0,1]=180.00
n = 1 	 t = 3 	 u[0,2]=1 	 y[0,2]=0 	 z[0,2]=0 	 p[0,2]=150.00
n = 1 	 t = 4 	 u[0,3]=1 	 y[0,3]=0 	 z[0,3]=0 	 p[0,3]=120.00
n = 1 	 t = 5 	 u[0,4]=1 	 y[0,4]=0 	 z[0,4]=0 	 p[0,4]=170.00
n = 1 	 t = 6 	 u[0,5]=1 	 y[0,5]=0 	 z[0,5]=0 	 p[0,5]=140.00
n = 2 	 t = 1 	 u[1,0]=1 	 y[1,0]=1 	 z[1,0]=0 	 p[1,0]=60.00
n = 2 	 t = 2 	 u[1,1]=1 	 y[1,1]=0 	 z[1,1]=0 	 p[1,1]=70.00
n = 2 	 t = 3 	 u[1,2]=1 	 y[1,2]=0 	 z[1,2]=0 	 p[1,2]=50.00
n = 2 	 t = 4 	 u[1,3]=1 	 y[1,3]=0 	 z[1,3]=0 	 p[1,3]=50.00
n = 2 	 t = 5 	 u[1,4]=1 	 y[1,4]=0 	 z[1,4]=0 	 p[1,4]=60.00
n = 2 	 t = 6 	 u[1,5]=1 	 y[1,5]=0 	 z[1,5]=0 	 p[1,5]=50.00
n = 3 	 t = 1 	 u[2,0]=0 	 y[2,0]=0 	 z[2,0]=0 	 p[2,0]=0.00
n = 3 	 t = 2 	 u[2,1]=0 	 y[2,1]=0 	 z[2,1]=0 	 p[2,1]=0.00
n = 3 	

In [ ]:
#Saving solution in a table and saving results in a CSV file

#Create two empty lists with the number of generators and periods
Gens = []
Periods = []
[(Gens.append(n+1), Periods.append(h+1))for n in N for h in T]
#[(Gens.append(n), Periods.append(h))for n in N for h in T]
#save results in dataframe
df_results = pd.DataFrame(zip(Gens, Periods, m.u[:,:](), m.y[:,:](), m.z[:,:](), m.p[:,:]()), 
                          columns=['Generator','Hour','On_Off','Startup','Shutdown','Power Generation'])

df_results.to_csv(path + '3a_uc_results.csv')  
df_results

,Generator,Hour,On_Off,Startup,Shutdown,Power Generation
0,1,1,1.0,0.0,0.0,180.0
1,1,2,1.0,0.0,0.0,180.0
2,1,3,1.0,0.0,0.0,150.0
3,1,4,1.0,0.0,0.0,120.0
4,1,5,1.0,0.0,0.0,170.0
5,1,6,1.0,0.0,0.0,140.0
6,2,1,1.0,1.0,0.0,60.0
7,2,2,1.0,0.0,0.0,70.0
8,2,3,1.0,0.0,0.0,50.0
9,2,4,1.0,0.0,0.0,50.0


## b.	
Using the solution to this UC calculate the reserves provided at each time period from each generator. In this calculation make sure to consider all of the generator’s constraints -including ramping limits-. i.e., you do not need to enforce ramp-feasibility of reserves in the UC formulation or solution, but account for this when calculating the reserves.  For which time periods are reserves provided above the reserve requirements? Should we always enforce ramp-feasibility of reserve requirements in the UC formulation and solution?

In [ ]:
# 1. In the dataframe with the info of generators add the id of generators 
df_gens['Generator'] = [1,2,3]

#2. Create a copy of dataframe with results and merge it with the generators 
# dataframe, only the generators id, max generation (capacity) and Ramp up limits
# values are needed
df_results_with_res = df_results.copy()
df_results_with_res = df_results_with_res.merge(df_gens.loc[:,['Generator', 'MaxGen', 'RampUp']], how='inner', on='Generator')
df_results_with_res

,Generator,Hour,On_Off,Startup,Shutdown,Power Generation,MaxGen,RampUp
0,1,1,1.0,0.0,0.0,180.0,300,50
1,1,2,1.0,0.0,0.0,180.0,300,50
2,1,3,1.0,0.0,0.0,150.0,300,50
3,1,4,1.0,0.0,0.0,120.0,300,50
4,1,5,1.0,0.0,0.0,170.0,300,50
5,1,6,1.0,0.0,0.0,140.0,300,50
6,2,1,1.0,1.0,0.0,60.0,200,60
7,2,2,1.0,0.0,0.0,70.0,200,60
8,2,3,1.0,0.0,0.0,50.0,200,60
9,2,4,1.0,0.0,0.0,50.0,200,60


In [ ]:
# 3. Estimate reserves provided 
def calculate_reserves_provided(row):
  ''' Calculates the provided reserves. 
  First, the available power is computed as the difference between the maximum
  capacity and the current power generation (for each period). 
  Then, the provided reserves are estimated as the minimum value between the 
  available reserves and the ramp up limits. 
  Finally, if the unit is offline, the reserves are equal to zero.'''
  power_available = row['MaxGen'] - row['Power Generation']
  reserves = np.minimum(power_available, row['RampUp']) * row['On_Off']
  return reserves


df_results_with_res['Reserves Provided'] = df_results_with_res.apply(calculate_reserves_provided, axis =1)
#Drop unnecesary columns to show the results
df_results_with_res.drop(columns=['MaxGen', 'RampUp'], inplace= True)
df_results_with_res.to_csv(path + '3b_uc_results.csv')
df_results_with_res

,Generator,Hour,On_Off,Startup,Shutdown,Power Generation,Reserves Provided
0,1,1,1.0,0.0,0.0,180.0,50.0
1,1,2,1.0,0.0,0.0,180.0,50.0
2,1,3,1.0,0.0,0.0,150.0,50.0
3,1,4,1.0,0.0,0.0,120.0,50.0
4,1,5,1.0,0.0,0.0,170.0,50.0
5,1,6,1.0,0.0,0.0,140.0,50.0
6,2,1,1.0,1.0,0.0,60.0,60.0
7,2,2,1.0,0.0,0.0,70.0,60.0
8,2,3,1.0,0.0,0.0,50.0,60.0
9,2,4,1.0,0.0,0.0,50.0,60.0


In [ ]:
%%shell
jupyter nbconvert --to html "/content/gdrive/MyDrive/ENV717-S2023/A06/ENV717_A6_Solution.ipynb"

[NbConvertApp] Converting notebook /content/gdrive/MyDrive/ENV717-S2023/A06/ENV717_A6_Solution.ipynb to html
[NbConvertApp] Writing 366278 bytes to /content/gdrive/MyDrive/ENV717-S2023/A06/ENV717_A6_Solution.html
